In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.optimize as opt

dt = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
c_dt, k_dt, w_dt, r_dt, y_dt = dt[:,0], dt[:,1], dt[:,2], dt[:,3], dt[:,4]

In [2]:
k_1 = k_dt.mean()

S = 1000
T = 100

# Random draw from Uniform(0,1)
sim = np.random.uniform(0,1,(T,S))

In [3]:
# Calculate simulated variables
def simulation(alpha, beta, rho, mu, sig, sim):
    epsilon = sts.norm.ppf(sim,0,sig)
    
    z = np.zeros((T,S))
    z_t_1 = mu
    for i in range(T):
        z_t = rho*z_t_1 + (1-rho)*mu + epsilon[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
    
    k = np.zeros((T,S))
    k_lag = np.zeros((T,S))
    k_t_1 = k_1
    for i in range(T):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_lag[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t

    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag - k
    y = np.exp(z)*k_lag**alpha
    
    return c,k_lag,k,w,r,y

In [4]:
# Data moment
mom1 = c_dt.mean()
mom2 = k_dt.mean()
mom3 = (c_dt/y_dt).mean()
mom4 = y_dt.std()
mom5 = np.corrcoef(c_dt[0:99],c_dt[1:100])[0,1]
mom6 = np.corrcoef(c_dt,k_dt)[0,1]
data_mom = np.array([mom1,mom2,mom3,mom4,mom5,mom6])

# Model moment
def model_mom(alpha, beta, rho, mu, sig, sim):
    c,k_lag,k,w,r,y = simulation(alpha, beta, rho, mu, sig, sim)
    mom1 = c.mean()
    mom2 = k.mean()
    mom3 = (c/y).mean()
    mom4 = np.std(y,axis=0).mean()
    corr_c = np.zeros((1,S))
    corr_ck = np.zeros((1,S))
    
    for i in range(S):
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    mom5 = corr_c.mean()
    mom6 = corr_ck.mean()
    model_mom = np.array([mom1,mom2,mom3,mom4,mom5,mom6])
    return model_mom

def err_vec(alpha, beta, rho, mu, sig, sim):
    model_mmt = model_mom(alpha, beta, rho, mu, sig, sim)
    return (model_mmt - data_mom)/data_mom

def crit(params, *args):
    alpha, rho, mu, sig = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sig, sim)
    W = np.eye(6)
    return err @ W @ err.T

In [5]:
alpha = 0.4
beta = 0.99
rho = 0.9
mu = 10
sig = 0.1

params_init = np.array([alpha, rho, mu, sig])
bnds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,None))

result = opt.minimize(crit, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bnds)
alpha_smm, rho_smm, mu_smm, sig_smm = result.x
print(result)

      fun: 4.3551441326819306e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.15424698e-04,  -2.61760566e-05,  -1.27648738e-05,
        -5.67882615e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 195
      nit: 32
   status: 0
  success: True
        x: array([ 0.42103804,  0.92392213,  9.92184497,  0.09303466])


In [7]:
print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sig_smm)

alpha SMM =  0.421038037087
rho SMM =  0.923922127492
mu SMM =  9.92184497471
sigma SMM =  0.0930346580621
